<a href="https://colab.research.google.com/github/benjaminbrown038/WeightAndBiases/blob/main/notebooks/wandb/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WANDB Experiments

- 5 Runs

- Project

#### Imports

In [ ]:
%%capture
!pip3 install wandb -qU
import wandb
import random
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T
wandb.login()

In [ ]:
total_runs = 5

In [ ]:
for run in range(total_runs):
  wandb.init(project="basic-intro", name=f"experiment_{run}", config={"learning_rate": 0.02,"architecture": "CNN","dataset": "CIFAR-100","epochs": 10})

In [ ]:
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset
      wandb.log({"acc": acc, "loss": loss})
  wandb.finish()

acc,▁▆▇█▇▇██
loss,█▅▁▃▂▁▁▂
acc,0.82123
loss,0.22939


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

#### Set up a Dataloader

In [ ]:
def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset,
                                         batch_size=batch_size,
                                         shuffle=True if is_train else False,
                                         pin_memory=True, num_workers=2)
    return loader



In [ ]:
def get_model(dropout):
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model



In [ ]:
def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

In [ ]:
def log_image_table(images, predicted, labels, probs):
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

In [ ]:
for _ in range(5):
    wandb.init(project="pytorch-intro",config={"epochs": 10,"batch_size": 128,"lr": 1e-3,"dropout": random.uniform(0.01, 0.80)})
    config = wandb.config
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)
    model = get_model(config.dropout)
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            example_ct += len(images)
            metrics = {"train/train_loss": train_loss,"train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,"train/example_ct": example_ct}
            if step + 1 < n_steps_per_epoch:
                wandb.log(metrics)
            step_ct += 1
        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))
        val_metrics = {"val/val_loss": val_loss,"val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})
        print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")
    wandb.summary['test_accuracy'] = 0.8
    wandb.finish()

100%|██████████| 9912422/9912422 [00:00<00:00, 72856762.02it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 42459058.47it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 20676175.97it/s]


...

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▄▃▃▃▂▃▃▁▂▂▂▁▂▂▃▂▁▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▄▅▆▆▇██▇█
val/val_loss,█▅▄▃▂▂▁▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.0854
val/val_accuracy,0.948
val/val_loss,0.1644


Train Loss: 0.468, Valid Loss: 0.330160, Accuracy: 0.91
Train Loss: 0.322, Valid Loss: 0.258501, Accuracy: 0.92
Train Loss: 0.365, Valid Loss: 0.236898, Accuracy: 0.92
Train Loss: 0.245, Valid Loss: 0.218299, Accuracy: 0.93
Train Loss: 0.189, Valid Loss: 0.200007, Accuracy: 0.94
Train Loss: 0.496, Valid Loss: 0.193974, Accuracy: 0.94
Train Loss: 0.189, Valid Loss: 0.187818, Accuracy: 0.94
Train Loss: 0.255, Valid Loss: 0.176592, Accuracy: 0.94
Train Loss: 0.174, Valid Loss: 0.178744, Accuracy: 0.94
Train Loss: 0.080, Valid Loss: 0.171949, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▄▃▃▂▂▃▁▃▂▂▂▂▂▂▁▂▂▂▁▂▁▂▂▂▁▂▁▁▂▁▁▂▂▁▁▁▁▁
val/val_accuracy,▁▄▄▅▆▆▆▇▇█
val/val_loss,█▅▄▃▂▂▂▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.08046
val/val_accuracy,0.949
val/val_loss,0.17195


Train Loss: 0.314, Valid Loss: 0.285400, Accuracy: 0.92
Train Loss: 0.236, Valid Loss: 0.232365, Accuracy: 0.93
Train Loss: 0.173, Valid Loss: 0.215804, Accuracy: 0.93
Train Loss: 0.198, Valid Loss: 0.192667, Accuracy: 0.94
Train Loss: 0.080, Valid Loss: 0.179237, Accuracy: 0.95
Train Loss: 0.060, Valid Loss: 0.172385, Accuracy: 0.95
Train Loss: 0.096, Valid Loss: 0.162351, Accuracy: 0.95
Train Loss: 0.042, Valid Loss: 0.154967, Accuracy: 0.95
Train Loss: 0.020, Valid Loss: 0.158260, Accuracy: 0.95
Train Loss: 0.115, Valid Loss: 0.153087, Accuracy: 0.96


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▃▃▂▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▃▃▅▇▇▆█▇█
val/val_loss,█▅▄▃▂▂▁▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.11517
val/val_accuracy,0.9555
val/val_loss,0.15309


In [ ]:
wandb.alert(title="Low accuracy",text=f"Accuracy is below the acceptable threshold")

UsageError: ignored

In [ ]:
wandb.init(project="pytorch-intro")

Accuracy is: 0.879, 0.3
Accuracy is: 1.788, 0.3
Accuracy is: 1.022, 0.3
Accuracy is: 0.679, 0.3
Accuracy is: 0.683, 0.3
Accuracy is: 0.432, 0.3
Accuracy is: 1.809, 0.3
Accuracy is: 1.147, 0.3
Accuracy is: 1.695, 0.3
Accuracy is: 0.332, 0.3
Accuracy is: 0.919, 0.3
Accuracy is: 1.214, 0.3
Accuracy is: 1.025, 0.3
Accuracy is: 0.503, 0.3
Accuracy is: 1.192, 0.3
Accuracy is: 1.136, 0.3
Accuracy is: 1.324, 0.3
Accuracy is: 0.726, 0.3
Accuracy is: 1.628, 0.3
Accuracy is: 0.294, 0.3
Alert triggered


Accuracy,▄█▄▃▃▂█▅▇▁▄▅▄▂▅▅▆▃▇▁
Accuracy,0.294


In [ ]:
acc_threshold = 0.3

In [ ]:
for training_step in range(1000):
    accuracy = round(random.random() + random.random(), 3)
    print(f'Accuracy is: {accuracy}, {acc_threshold}')
    wandb.log({"Accuracy": accuracy})
    if accuracy <= acc_threshold:
        wandb.alert(title='Low Accuracy',text=f'Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}',)
        print('Alert triggered')
        break

In [ ]:
wandb.finish()

In [ ]:
wandb.alert(title="Low accuracy",text=f"Accuracy is below the acceptable threshold")

UsageError: ignored